In [1]:
import os
import modules.util as util
import math
import numpy as np
import modules.datagenerator as dtgen
import modules.network_PoolerCLS as network
import modules.loss as loss
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from tensorflow import keras

2023-03-09 00:57:09.512047: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
home=os.path.abspath(os.getcwd())
data_path=os.path.join(home, 'data_train_valset')

#global data
pairs=[]
classes=[]

for file in os.listdir(data_path):
    classes.append(file)

In [3]:
util.make_pairs(data_path, pairs, classes)

# partition = {'train': np.arange(math.floor(len(pairs)*.6)),
#              'validation': np.arange(math.floor(len(pairs)*.6),math.floor(len(pairs)*.8)),
#              'test': np.arange(math.floor(len(pairs)*.8), math.floor(len(pairs)))}

partition = {'train': np.arange(math.floor(len(pairs)*.75)),
             'validation': np.arange(math.floor(len(pairs)*.75),math.floor(len(pairs)))}

# Generators
train_generator = dtgen.DataGenerator(partition['train'], pairs, batch_size=16)
val_generator = dtgen.DataGenerator(partition['validation'], pairs, batch_size=16)

In [4]:
len(pairs)

5146

In [5]:
len(partition["train"])

3859

In [6]:
len(partition["validation"])

1287

In [7]:
partition

{'train': array([   0,    1,    2, ..., 3856, 3857, 3858]),
 'validation': array([3859, 3860, 3861, ..., 5143, 5144, 5145])}

In [8]:
siamese_obj = network.SiameseNetwork()
siamese_network = siamese_obj.make_siamese_net()

All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [9]:
siamese_network.summary()

Model: "siamese_network"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 pixel_values1 (InputLayer)     [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 pixel_values2 (InputLayer)     [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 permute (Permute)              (None, 3, 224, 224)  0           ['pixel_values1[0][0]']          
                                                                                    

In [10]:
siamese_network.compile(loss= loss.loss(1), optimizer=tf.keras.optimizers.Adam(learning_rate=0.5e-3), # 0.5e-6
                    metrics=["accuracy"])

In [11]:
checkpoint_path = os.path.join(home, 'checkpoints')
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', 
                             verbose=1, save_best_only=True, mode='max')
early_callback=tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=35, restore_best_weights=True)

In [12]:
# callbacks_list = [early_callback,checkpoint]
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        lr = lr * tf.math.exp(-0.05)
        if lr >= 1e-8:
            return lr
        else:
            return 1e-8
# 0.5e-7

class print_lr(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(keras.backend.eval(self.model.optimizer.lr))

lr_sched = LearningRateScheduler(scheduler)

callbacks_list = [early_callback, lr_sched, checkpoint, print_lr()]

In [13]:
# Fit the model
# siamese_net = keras.models.load_model('checkpoints', custom_objects={ 'contrastive_loss': loss(1) })

batch_size=32
epochs=500
history = siamese_network.fit(
    train_generator,
    validation_data=val_generator,
    callbacks=callbacks_list,
    epochs=epochs,
)
# history = siamese_net.fit(
#     [x_train_1, x_train_2],
#     y_train,
#     validation_data=([x_val_1, x_val_2], y_val),
#     batch_size=batch_size,
#     callbacks=callbacks_list,
#     epochs=epochs,
#     # sample_weight=sample_weight,
# )

print(max(history.history["val_accuracy"]))

Epoch 1/500


2023-03-09 00:58:45.802489: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 28.42MiB (rounded to 29805312)requested by op siamese_network/vit/encoder/layer_._5/attention/attention/Softmax
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-03-09 00:58:45.805160: W tensorflow/tsl/framework/bfc_allocator.cc:492] ****************************************************************************************************
2023-03-09 00:58:45.805237: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at softmax_op_gpu.cu.cc:222 : RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[16,12,197,197] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc


RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL ResourceExhaustedError REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /home/halim/anaconda3/envs/ViT-setup/lib/python3.10/site-packages/tensorflow/python/framework/errors_impl.py(372): __init__
  /home/halim/anaconda3/envs/ViT-setup/lib/python3.10/site-packages/tensorflow/python/eager/execute.py(52): quick_execute
  /home/halim/anaconda3/envs/ViT-setup/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_function/monomorphic_function.py(378): call
  /home/halim/anaconda3/envs/ViT-setup/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_function/monomorphic_function.py(1745): _call_flat
  /home/halim/anaconda3/envs/ViT-setup/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_function/tracing_compiler.py(134): __call__
  /home/halim/anaconda3/envs/ViT-setup/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_function/p